# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
geoapify_key

'a03d6926da2e42ed903565d70a851fc8'

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust-nera,64.5667,143.2000,1.23,82,91,1.41,RU,1714866765
1,1,adamstown,-25.0660,-130.1015,21.19,65,20,8.78,PN,1714866757
2,2,grytviken,-54.2811,-36.5092,0.24,78,16,5.03,GS,1714866959
3,3,west island,-12.1568,96.8225,27.99,78,100,7.72,CC,1714866761
4,4,nikol'sk,53.7189,46.0712,0.46,71,41,3.19,RU,1714867012


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [48]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color= "City")

# Display the map plot_1
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [113]:
# Narrow down cities that fit criteria and drop any results with null values
# Ideal weather conditions temp & humidity
max_temp = 30 
min_temp = 20 
max_humidity = 50
min_humidity = 20
max_cloud = 50 
max_wind_speed = 6

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= min_temp) & 
                                (city_data_df['Max Temp'] <= max_temp) & 
                                (city_data_df['Humidity'] <= max_humidity) & 
                                (city_data_df['Humidity'] >= min_humidity) & 
                                (city_data_df['Cloudiness'] <= max_cloud) &
                                (city_data_df['Wind Speed'] <= max_wind_speed)].copy()

# Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# Display sample data
print(len(ideal_weather_df))
ideal_weather_df.head(20)

15


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,ixtapa,20.7000,-105.2000,29.87,48,37,2.49,MX,1714867273
57,57,alvorada,-12.4800,-49.1247,25.49,48,6,1.50,BR,1714867280
88,88,douz,33.4663,9.0203,21.27,46,0,4.66,TN,1714867032
176,176,nalut,30.3333,10.8500,21.67,33,0,5.75,LY,1714867316
271,271,al kharj,24.1550,47.3346,26.32,32,0,4.39,SA,1714867346
273,273,mount isa,-20.7333,139.5000,22.87,40,35,5.14,AU,1714867346
289,289,as samawah,31.3320,45.2944,26.52,24,0,2.64,IQ,1714867351
346,346,pindi bhattian,31.8978,73.2730,23.90,43,1,2.49,PK,1714867368
388,388,port hedland,-20.3167,118.5667,20.03,49,0,2.57,AU,1714867382
430,430,bahir dar,11.5936,37.3908,21.83,22,25,2.73,ET,1714867395


### Step 3: Create a new DataFrame called `hotel_df`.

In [114]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
34,ixtapa,MX,20.7000,-105.2000,48,
57,alvorada,BR,-12.4800,-49.1247,48,
88,douz,TN,33.4663,9.0203,46,
176,nalut,LY,30.3333,10.8500,33,
271,al kharj,SA,24.1550,47.3346,32,
273,mount isa,AU,-20.7333,139.5000,40,
289,as samawah,IQ,31.3320,45.2944,24,
346,pindi bhattian,PK,31.8978,73.2730,43,
388,port hedland,AU,-20.3167,118.5667,49,
430,bahir dar,ET,11.5936,37.3908,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [115]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 3

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
 
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ixtapa - nearest hotel: Ma. Cristina
alvorada - nearest hotel: No hotel found
douz - nearest hotel: Louage station Douz
nalut - nearest hotel: No hotel found
al kharj - nearest hotel: شقق فندقية
mount isa - nearest hotel: Ibis Styles
as samawah - nearest hotel: فندق قصر الغدير
pindi bhattian - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
bahir dar - nearest hotel: ናኪ ሆቴል
hit - nearest hotel: No hotel found
iiyama - nearest hotel: No hotel found
caldwell - nearest hotel: Best Western Plus Caldwell Inn & Suites
colgong - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,ixtapa,MX,20.7000,-105.2000,48,Ma. Cristina
57,alvorada,BR,-12.4800,-49.1247,48,No hotel found
88,douz,TN,33.4663,9.0203,46,Louage station Douz
176,nalut,LY,30.3333,10.8500,33,No hotel found
271,al kharj,SA,24.1550,47.3346,32,شقق فندقية
273,mount isa,AU,-20.7333,139.5000,40,Ibis Styles
289,as samawah,IQ,31.3320,45.2944,24,فندق قصر الغدير
346,pindi bhattian,PK,31.8978,73.2730,43,No hotel found
388,port hedland,AU,-20.3167,118.5667,49,The Esplanade Hotel
430,bahir dar,ET,11.5936,37.3908,22,ናኪ ሆቴል


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [128]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color= "City",
    hover_cols=["Country", "Hotel Name"])

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)